In [1]:
import pandas as pd
import pymysql
from pyhive import hive
import pandas as pd
from pyspark import SparkContext,SQLContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
import datetime

In [2]:
class hiveParse(object):
    """
    该方法用于读取hive 数据，并转化成pandas dataframe
    """
    def __init__(self, host='', port=10000, user='', passwd='', dbName=''):
        self.host = host
        self.port = port
        self.user = user
        self.passwd = passwd
        self.dbName = dbName

    def conn_hive(self):
        self.connection = hive.Connection(host=self.host,port=self.port,username=self.user)

    def hive_connect(self,sql_select):
        cur = self.connection.cursor()
        cur.execute(sql_select)
        columns = [col[0] for col in cur.description]
        data = cur.fetchall()
        query_df = pd.DataFrame(data)
        query_df.columns = columns
        return query_df

class mysqldbParse(object):
    """
    该方法用于连接mysql，并执行相应操作
    """

    def __init__(self, host='', port=3306, user='', passwd='', dbName='', charset='utf8', connect_timeout=31536000):
        self.host = host
        self.port = port
        self.user = user
        self.passwd = passwd
        self.dbName = dbName
        self.charset = charset
        self.connect_timeout = connect_timeout

    def conn_mysql(self):
        self.conn = pymysql.connect(host=self.host,
                                    port=self.port,
                                    user=self.user,
                                    password=self.passwd,
                                    db=self.dbName,
                                    charset=self.charset,
                                    connect_timeout=self.connect_timeout,
                                    cursorclass=pymysql.cursors.DictCursor)

    def insert_sql(self, insertSql):
        with self.conn.cursor() as cursor:
            cursor.execute(insertSql)
            self.conn.commit()

    def select_Sql(self, selectSql):
        df = pd.read_sql(selectSql, self.conn)
        return df

    def close(self):
        self.conn.close()

In [3]:
def hive_df(sql):
    host = "58.59.18.61"
    port = 10000
    user = "hdfs"
    dbName = "ods"
    hive_parse = hiveParse(host,port,user,dbName)
    hive_parse.conn_hive()
    hive_pd = hive_parse.hive_connect(sql)
    return hive_pd

def mysql_df(selectSql):
    host = "58.59.11.86"
    port = 3306
    user = "tangdinghai"
    passwd = "Tangdinghai2019!"
    dbName = "reportpublic"
    mysql_parse = mysqldbParse(host,port,user,passwd,dbName)
    mysql_parse.conn_mysql()
    mysql_pd = mysql_parse.select_Sql(selectSql)
    mysql_parse.close()
    return mysql_pd



In [4]:
#### 有授信额度相关客户级和订单级汇总监控报表---日报

In [5]:
mysql_sql =  """

select
	score_flag as score_interval,
	count(distinct(user_id)) as shouxin_uid_%s,
    count(distinct(CREDIT_APPLY_ID)) as shouxin_applyid_%s

from
(
	select
		user_id,
        CREDIT_APPLY_ID,
        create_time,
		case
			when (t.score ) > 680 then ">680"
			when (t.score ) > 650 and (t.score ) <= 680 then "(650,680]"
			when (t.score ) > 630 and (t.score ) <= 650 then "(630,650]"
			when (t.score ) > 610 and (t.score ) <= 630 then "(610,630]"
			when (t.score ) > 595 and (t.score ) <= 610 then "(595,610]"
			when (t.score ) <= 595 then "<=595"
		end as score_flag
	from 
    
             (
                select 
                        b.USER_ID as user_id,
                        b.CREDIT_APPLY_ID as CREDIT_APPLY_ID,
                        b.APPLY_STATUS as APPLY_STATUS,
                        b.create_time as create_time,
                        a.SCORE as score
                from
                (
                    select 
                            USER_ID,
                            CREDIT_ID,
                            CREDIT_APPLY_ID,
                            CREDIT_AMOUNT,
                            SCORE,
                            PRODUCT_NO
                    from ods_credit_credit_info where product_no like 'PN00000053%%'
                    ) as a 

                    right join
                    (
                    select 
                            TENANT_ID,
                            USER_ID,
                            CREDIT_APPLY_ID,
                            APPLY_STATUS,
                            create_time
                    from ods_credit_credit_apply
                    where product_no like 'PN00000053%%'
                    ) as b
                    on a.USER_ID = b.USER_ID
                ) as t
                  where date(t.create_time) 
                  between %s  and %s  -- 今天减去一天
) as k group by k.score_flag


"""
# table_day = mysql_df(sql_day)

In [6]:
def dairly_monitor_shouxin(sql,flag,daydiff1=0,daydiff2=0):
    
    if(flag == 'today'):
        time1 = """ADDDATE(curdate(),-1)"""
        time2 = """ADDDATE(curdate(),-1)"""
        flag = 'today'
        excute_sql = sql%(flag,flag,time1,time2)
#         print(excute_sql)
    if(flag == 'week'):
        time1 = """ADDDATE(curdate(),-%d)"""%daydiff1
        time2 = """ADDDATE(curdate(),-%d)"""%daydiff2
        excute_sql = sql%(flag,flag,time1,time2)
        
    if(flag == 'month'):
        time1 = """ADDDATE(ADDDATE(curdate(),INTERVAL -%d month),-%d)"""%(daydiff1,daydiff2)
        time2 = """ADDDATE(curdate(),-%d)"""%daydiff2
        excute_sql = sql%(flag,flag,time1,time2)
    #获取每个月的时间
    credit_apply = mysql_df(excute_sql)
    return credit_apply

In [7]:
daily_shouxin_day = dairly_monitor_shouxin(mysql_sql,'today',0,0)
daily_shouxin_week = dairly_monitor_shouxin(mysql_sql,'week',8,2)
daily_shouxin_month = dairly_monitor_shouxin(mysql_sql,'month',1,2)



base_dict = {'score_interval':['>680','(650,680]','(630,650]','(610,630]','(595,610]','<=595','汇总'],\
             'toutiao_develop':['24.89%','20.10%','17.00%','14.87%','14.29%','10.85%','21590']}
dairy_report = pd.DataFrame(base_dict)


shouxin_monitor_dairly = dairy_report.merge(daily_shouxin_month,on = 'score_interval',how='left')\
                                  .merge(daily_shouxin_week,on = 'score_interval',how='left')\
                                  .merge(daily_shouxin_day,on = 'score_interval',how='left').fillna(0)


col_keep = list(shouxin_monitor_dairly.columns)
col_keep.remove('score_interval')
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(shouxin_monitor_dairly[col])
    shouxin_monitor_dairly[col][6] = col_total
    shouxin_monitor_dairly[col+"_percent"] = shouxin_monitor_dairly[col].map(lambda x: x/(col_total+0.000001))
    shouxin_monitor_dairly[col+"_percent"][6] = col_total 

    
shouxin_monitor_dairly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,score_interval,toutiao_develop,shouxin_uid_month,shouxin_applyid_month,shouxin_uid_week,shouxin_applyid_week,shouxin_uid_today,shouxin_applyid_today,shouxin_uid_month_percent,shouxin_applyid_month_percent,shouxin_uid_week_percent,shouxin_applyid_week_percent,shouxin_uid_today_percent,shouxin_applyid_today_percent
0,>680,24.89%,4.0,4.0,2.0,2.0,0.0,0.0,0.028369,0.028369,0.022727,0.022727,0.0,0.0
1,"(650,680]",20.10%,40.0,40.0,20.0,20.0,0.0,0.0,0.283688,0.283688,0.227273,0.227273,0.0,0.0
2,"(630,650]",17.00%,32.0,32.0,23.0,23.0,0.0,0.0,0.226950,0.226950,0.261364,0.261364,0.0,0.0
3,"(610,630]",14.87%,45.0,45.0,25.0,25.0,0.0,0.0,0.319149,0.319149,0.284091,0.284091,0.0,0.0
4,"(595,610]",14.29%,18.0,18.0,17.0,17.0,0.0,0.0,0.127660,0.127660,0.193182,0.193182,0.0,0.0
5,<=595,10.85%,2.0,2.0,1.0,1.0,0.0,0.0,0.014184,0.014184,0.011364,0.011364,0.0,0.0
6,汇总,21590,141.0,141.0,88.0,88.0,0.0,0.0,141.000000,141.000000,88.000000,88.000000,0.0,0.0


In [8]:
spark=SparkSession \
        .builder \
        .config("spark.eventLog.enabled", "false") \
        .config("spark.executor.memory", "4g")\
        .config("spark.driver.memory", "8g")\
        .config("spark.cores.max", "10")\
        .config("spark.task.maxFailures", "1000")\
        .config("spark.default.parallelism", "500")\
        .config("spark.sql.shuffle.partitions",50)\
        .appName('renhang_etl') \
        .master('yarn')\
        .getOrCreate()




In [9]:
spark_df = spark.createDataFrame(application_monitor).createOrReplaceTempView("application_monitor")

NameError: name 'application_monitor' is not defined

In [ ]:
import datetime
now_time = (datetime.datetime.now()+datetime.timedelta(days=-1)).strftime('%Y-%m-%d')
now_time


sql =  """

insert overwrite table renhang_user_profile.apply_monitor_user_day partition(dt='%s') 
select 
    score_interval,
    toutiao_develop,
    apply_day_percent,
    apply_week_percent,
    apply_month_percent
from application_monitor

""" %now_time

spark.sql(sql)


In [10]:
###########################################
#### 有授信额度相关客户级和订单级汇总监控报表---月报

In [11]:
# my_sql =  """

# select
# 	score_flag as score_interval,
# 	count(distinct(user_id)) as apply_num_%s
# from
# (
# 	select
# 		user_id,
#         create_time,
# 		case
# 			when (t.score ) > 680 then ">680"
# 			when (t.score ) > 650 and (t.score ) <= 680 then "(650,680]"
# 			when (t.score ) > 630 and (t.score ) <= 650 then "(630,650]"
# 			when (t.score ) > 610 and (t.score ) <= 630 then "(610,630]"
# 			when (t.score ) > 595 and (t.score ) <= 610 then "(595,610]"
# 			when (t.score ) <= 595 then "<=595"
# 		end as score_flag
# 	from 
    
#              (
#                 select 
#                         b.USER_ID as user_id,
#                         b.APPLY_STATUS as APPLY_STATUS,
#                         b.create_time as create_time,
#                         a.SCORE as score
#                 from
#                 (
#                     select 
#                             USER_ID,
#                             CREDIT_ID,
#                             CREDIT_APPLY_ID,
#                             CREDIT_AMOUNT,
#                             SCORE,
#                             PRODUCT_NO
#                     from ods_credit_credit_info where product_no like 'PN00000053%%'
#                     ) as a 

#                     right join
#                     (
#                     select 
#                             TENANT_ID,
#                             USER_ID,
#                             CREDIT_APPLY_ID,
#                             APPLY_STATUS,
#                             create_time
#                     from ods_credit_credit_apply
#                     where product_no like 'PN00000053%%'
#                     ) as b
#                     on a.USER_ID = b.USER_ID
#                 ) as t
#                   where date(t.create_time) between 
#                     %s and %s

# ) as k group by k.score_flag

# """
# # credit_apply = mysql_df(sql)

In [12]:
def monthly_monitor_shouxin(sql,flag,monthdiff):
    
    if(flag == 'online'):
        time1 = """ADDDATE(DATE_ADD(STR_TO_DATE('2019-01-14',"%Y-%m-%d"),interval -day(STR_TO_DATE('2019-01-14',"%Y-%m-%d"))+1 day),  interval -0 month)"""
        time2 = """last_day(STR_TO_DATE('2019-01-14',"%Y-%m-%d"))"""
        flag = 'online'
        excute_sql = sql%(flag,flag,time1,time2)

    else:
        time1 = """ADDDATE(DATE_ADD(curdate(),interval -day(curdate())+1 day),  interval -%d month)"""%monthdiff
        time2 = """ADDDATE(last_day(curdate()),interval -%d month)"""%monthdiff
        date_sql = """select ADDDATE(last_day(curdate()),interval -%d month) as date"""%monthdiff
        date =  mysql_df(date_sql)['date'].astype('str')
        flag = date[0][0:7].replace('-','_')
        excute_sql = sql%(flag,flag,time1,time2)

    #获取每个月的时间
    credit_apply = mysql_df(excute_sql)
    return credit_apply

In [13]:
#上线月
shouxin_month_online = monthly_monitor_shouxin(mysql_sql,'online',-1)
#当月
shouxin_month_now = monthly_monitor_shouxin(mysql_sql,'not_online',0)
#上月
shouxin_month_last1 = monthly_monitor_shouxin(mysql_sql,'not_online',1)
#上两月
shouxin_month_last2 = monthly_monitor_shouxin(mysql_sql,'not_online',2)
#上3月
shouxin_month_last3 = monthly_monitor_shouxin(mysql_sql,'not_online',3)
#上4月
shouxin_month_last4 = monthly_monitor_shouxin(mysql_sql,'not_online',4)
#上5月
shouxin_month_last5 = monthly_monitor_shouxin(mysql_sql,'not_online',5)
#上6月
shouxin_month_last6 = monthly_monitor_shouxin(mysql_sql,'not_online',6)


In [14]:
shouxin_monitor_monthly = dairy_report.merge(shouxin_month_online,on = 'score_interval',how='left')\
                                  .merge(shouxin_month_last6,on = 'score_interval',how='left')\
                                  .merge(shouxin_month_last5,on = 'score_interval',how='left')\
                                  .merge(shouxin_month_last4,on = 'score_interval',how='left')\
                                  .merge(shouxin_month_last3,on = 'score_interval',how='left')\
                                  .merge(shouxin_month_last2,on = 'score_interval',how='left')\
                                  .merge(shouxin_month_last1,on = 'score_interval',how='left')\
                                  .merge(shouxin_month_now,on = 'score_interval',how='left').fillna(0)

col_keep = list(shouxin_monitor_monthly.columns)
col_keep.remove('score_interval')
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(shouxin_monitor_monthly[col])
    shouxin_monitor_monthly[col][6] = col_total
    shouxin_monitor_monthly[col+"_percent"] = shouxin_monitor_monthly[col].map(lambda x: x/(col_total+0.000001))
    shouxin_monitor_monthly[col+"_percent"][6] = col_total 

    
shouxin_monitor_monthly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,score_interval,toutiao_develop,shouxin_uid_online,shouxin_applyid_online,shouxin_uid_2018_07,shouxin_applyid_2018_07,shouxin_uid_2018_08,shouxin_applyid_2018_08,shouxin_uid_2018_09,shouxin_applyid_2018_09,...,shouxin_uid_2018_09_percent,shouxin_applyid_2018_09_percent,shouxin_uid_2018_10_percent,shouxin_applyid_2018_10_percent,shouxin_uid_2018_11_percent,shouxin_applyid_2018_11_percent,shouxin_uid_2018_12_percent,shouxin_applyid_2018_12_percent,shouxin_uid_2019_01_percent,shouxin_applyid_2019_01_percent
0,>680,24.89%,4.0,4.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028369,0.028369
1,"(650,680]",20.10%,40.0,40.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.283688,0.283688
2,"(630,650]",17.00%,32.0,32.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.226950,0.226950
3,"(610,630]",14.87%,45.0,45.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.319149,0.319149
4,"(595,610]",14.29%,18.0,18.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.127660,0.127660
5,<=595,10.85%,2.0,2.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014184,0.014184
6,汇总,21590,141.0,141.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.000000,141.000000


In [15]:
#### 申请监控订单级和客户级日报 --- from hive

In [16]:
hive_sql = """


select
	score_interval,
	count(distinct(user_id)) as apply_uid_%s,
    count(distinct(session_id)) as apply_sessionId_%s
from
(
	select
		user_id,
        session_id,
		case
			when cast(t.out_credit_score as int) > 680 then ">680"
			when cast(t.out_credit_score as int) > 650 and cast(t.out_credit_score as int) <= 680 then "(650,680]"
			when cast(t.out_credit_score as int) > 630 and cast(t.out_credit_score as int) <= 650 then "(630,650]"
			when cast(t.out_credit_score as int) > 610 and cast(t.out_credit_score as int) <= 630 then "(610,630]"
			when cast(t.out_credit_score as int) > 595 and cast(t.out_credit_score as int) <= 610 then "(595,610]"
			when cast(t.out_credit_score as int) <= 595 then "<=595"
		end as score_interval
	from 
			(
				SELECT
     				a.session_id as session_id,
     				a.user_id as user_id,
     				a.t02_apply_time as apply_time,
     				b.out_credit_score as out_credit_score
				FROM 
				(
					SELECT 
						session_id,
       					user_id,
       					t02_apply_time
					FROM ods.ods_toutiao_shouxin_input 
				) AS a

			left JOIN
  			(
  				SELECT 
  					session_id,
  					user_id,
          			out_credit_score
   				FROM ods.ods_toutiao_shouxin_output
  			) AS b ON a.session_id = b.session_id
		) as t  where from_unixtime(unix_timestamp(substr(t.apply_time,0,8),'yyyymmdd'),'yyyy-mm-dd') 
            between %s and %s  
) as a group by a.score_interval

"""
# toutiao = hive_df(hive_sql)

In [17]:
def dairly_monitor_apply(sql,flag,daydiff1=0,daydiff2=0):
    
    if(flag == 'today'):
        time1 = """date_add(CURRENT_DATE,-1)"""
        time2 = """date_add(CURRENT_DATE,-1)"""
        flag = 'today'
        excute_sql = sql%(flag,flag,time1,time2)
#         print(excute_sql)
    if(flag == 'week'):
        time1 = """date_add(CURRENT_DATE,-%d)"""%daydiff1
        time2 = """date_add(CURRENT_DATE,-%d)"""%daydiff2
        excute_sql = sql%(flag,flag,time1,time2)
        
    if(flag == 'month'):
        time1 = """date_add(add_months(CURRENT_DATE, -%d ),-%d)"""%(daydiff1,daydiff2)
        time2 = """date_add(CURRENT_DATE,-%d)"""%daydiff2
        excute_sql = sql%(flag,flag,time1,time2)
    #获取每个月的时间
    credit_apply = hive_df(excute_sql)
    return credit_apply

In [18]:
dairly_apply_day = dairly_monitor_apply(hive_sql,'today',0,0)
dairly_apply_week = dairly_monitor_apply(hive_sql,'week',8,2)
dairly_apply_month = dairly_monitor_apply(hive_sql,'month',1,2)


# 固定的两列
base_dict = {'score_interval':['>680','(650,680]','(630,650]','(610,630]','(595,610]','<=595','汇总'],\
             'toutiao_develop':['24.89%','20.10%','17.00%','14.87%','14.29%','10.85%','21590']}
dairy_report = pd.DataFrame(base_dict)


apply_monitor_dairly = dairy_report.merge(dairly_apply_month,on = 'score_interval',how='left')\
                                  .merge(dairly_apply_week,on = 'score_interval',how='left')\
                                  .merge(dairly_apply_day,on = 'score_interval',how='left').fillna(0)

col_keep = list(apply_monitor_dairly.columns)
col_keep.remove('score_interval')
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(apply_monitor_dairly[col])
    apply_monitor_dairly[col][6] = col_total
    apply_monitor_dairly[col+"_percent"] = apply_monitor_dairly[col].map(lambda x: x/(col_total+0.000001))
    apply_monitor_dairly[col+"_percent"][6] = col_total 

    
apply_monitor_dairly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,score_interval,toutiao_develop,apply_uid_month,apply_sessionid_month,apply_uid_week,apply_sessionid_week,apply_uid_today,apply_sessionid_today,apply_uid_month_percent,apply_sessionid_month_percent,apply_uid_week_percent,apply_sessionid_week_percent,apply_uid_today_percent,apply_sessionid_today_percent
0,>680,24.89%,10.0,10.0,8.0,8.0,0.0,0.0,0.007616,0.007610,0.008840,0.008840,0.000000,0.000000
1,"(650,680]",20.10%,66.0,66.0,35.0,35.0,4.0,4.0,0.050267,0.050228,0.038674,0.038674,0.142857,0.142857
2,"(630,650]",17.00%,79.0,79.0,41.0,41.0,3.0,3.0,0.060168,0.060122,0.045304,0.045304,0.107143,0.107143
3,"(610,630]",14.87%,149.0,149.0,76.0,76.0,3.0,3.0,0.113481,0.113394,0.083978,0.083978,0.107143,0.107143
4,"(595,610]",14.29%,187.0,187.0,128.0,128.0,4.0,4.0,0.142422,0.142314,0.141436,0.141436,0.142857,0.142857
5,<=595,10.85%,822.0,823.0,617.0,617.0,14.0,14.0,0.626047,0.626332,0.681768,0.681768,0.500000,0.500000
6,汇总,21590,1313.0,1314.0,905.0,905.0,28.0,28.0,1313.000000,1314.000000,905.000000,905.000000,28.000000,28.000000


In [19]:
#### 申请监控月报 --- from hive

In [20]:
hive_sql = """


select
	score_interval,
	count(distinct(user_id)) as apply_uid_%s,
    count(distinct(session_id)) as apply_sessionId_%s

from
(
	select
		user_id,
        session_id,
		case
			when cast(t.out_credit_score as int) > 680 then ">680"
			when cast(t.out_credit_score as int) > 650 and cast(t.out_credit_score as int) <= 680 then "(650,680]"
			when cast(t.out_credit_score as int) > 630 and cast(t.out_credit_score as int) <= 650 then "(630,650]"
			when cast(t.out_credit_score as int) > 610 and cast(t.out_credit_score as int) <= 630 then "(610,630]"
			when cast(t.out_credit_score as int) > 595 and cast(t.out_credit_score as int) <= 610 then "(595,610]"
			when cast(t.out_credit_score as int) <= 595 then "<=595"
		end as score_interval
	from 
			(
				SELECT
     				a.session_id as session_id,
     				a.user_id as user_id,
     				a.t02_apply_time as apply_time,
     				b.out_credit_score as out_credit_score
				FROM 
				(
					SELECT 
						session_id,
       					user_id,
       					t02_apply_time
					FROM ods.ods_toutiao_shouxin_input 
				) AS a

			left JOIN
  			(
  				SELECT 
  					session_id,
  					user_id,
          			out_credit_score
   				FROM ods.ods_toutiao_shouxin_output
  			) AS b ON a.session_id = b.session_id
		) as t  where from_unixtime(unix_timestamp(substr(t.apply_time,0,8),'yyyymmdd'),'yyyy-mm-dd') 
            between %s and %s  
) as a group by a.score_interval

"""
# toutiao = hive_df(hive_sql)

In [21]:
def monthly_monitor_apply(sql,flag,monthdiff):
    
    if(flag == 'online'):
        time1 = """date_add(last_day(add_months('2019-01-14', -1 )),1)"""
        time2 = """last_day('2019-01-14')"""
        flag = 'online'
        excute_sql = sql%(flag,flag,time1,time2)
#         print(excute_sql)
    else:
        time1 = """date_add(last_day(add_months(CURRENT_DATE, -%d )),1)"""%(monthdiff+1)    #当月第一天：上月最后一天+1天
        time2 = """last_day(add_months(CURRENT_DATE,-%d))"""%monthdiff                  #当月最后一天
        
        date_sql = """select last_day(add_months(CURRENT_DATE,-%d)) as dt"""%monthdiff
        date =  hive_df(date_sql)['dt'].astype('str')
        flag = date[0][0:7].replace('-','_')
        excute_sql = sql%(flag,flag,time1,time2)
#         print(excute_sql)

    #获取每个月的时间
    credit_apply = spark.sql(excute_sql).toPandas()
    return credit_apply

In [22]:
#上线月
apply_month_online = monthly_monitor_apply(hive_sql,'online',-1)
#当月
apply_month_now = monthly_monitor_apply(hive_sql,'not_online',0)
#上月
apply_month_last1 = monthly_monitor_apply(hive_sql,'not_online',1)
#上两月
apply_month_last2 = monthly_monitor_apply(hive_sql,'not_online',2)
#上3月
apply_month_last3 = monthly_monitor_apply(hive_sql,'not_online',3)
#上4月
apply_month_last4 = monthly_monitor_apply(hive_sql,'not_online',4)
#上5月
apply_month_last5 = monthly_monitor_apply(hive_sql,'not_online',5)
#上6月
apply_month_last6 = monthly_monitor_apply(hive_sql,'not_online',6)


In [23]:
base_dict = {'score_interval':['>680','(650,680]','(630,650]','(610,630]','(595,610]','<=595','汇总'],\
             'toutiao_develop':['24.89%','20.10%','17.00%','14.87%','14.29%','10.85%','21590']}
dairy_report = pd.DataFrame(base_dict)

apply_monitor_monthly = dairy_report.merge(apply_month_online,on = 'score_interval',how='left')\
                                  .merge(apply_month_last6,on = 'score_interval',how='left')\
                                  .merge(apply_month_last5,on = 'score_interval',how='left')\
                                  .merge(apply_month_last4,on = 'score_interval',how='left')\
                                  .merge(apply_month_last3,on = 'score_interval',how='left')\
                                  .merge(apply_month_last2,on = 'score_interval',how='left')\
                                  .merge(apply_month_last1,on = 'score_interval',how='left')\
                                  .merge(apply_month_now,on = 'score_interval',how='left').fillna(0)

col_keep = list(apply_monitor_monthly.columns)
col_keep.remove('score_interval')
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(apply_monitor_monthly[col])
    apply_monitor_monthly[col][6] = col_total
    apply_monitor_monthly[col+"_percent"] = apply_monitor_monthly[col].map(lambda x: x/(col_total+0.000001))
    apply_monitor_monthly[col+"_percent"][6] = col_total 

    
apply_monitor_monthly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,score_interval,toutiao_develop,apply_uid_online,apply_sessionId_online,apply_uid_2018_07,apply_sessionId_2018_07,apply_uid_2018_08,apply_sessionId_2018_08,apply_uid_2018_09,apply_sessionId_2018_09,...,apply_uid_2018_09_percent,apply_sessionId_2018_09_percent,apply_uid_2018_10_percent,apply_sessionId_2018_10_percent,apply_uid_2018_11_percent,apply_sessionId_2018_11_percent,apply_uid_2018_12_percent,apply_sessionId_2018_12_percent,apply_uid_2019_01_percent,apply_sessionId_2019_01_percent
0,>680,24.89%,10.0,10.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007457,0.007452
1,"(650,680]",20.10%,70.0,70.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052200,0.052161
2,"(630,650]",17.00%,82.0,82.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.061148,0.061103
3,"(610,630]",14.87%,152.0,152.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.113348,0.113264
4,"(595,610]",14.29%,191.0,191.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142431,0.142325
5,<=595,10.85%,836.0,837.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.623415,0.623696
6,汇总,21590,1341.0,1342.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1341.000000,1342.000000


In [24]:
####上述已经完成订单级汇总和客户级汇总
#### 申请表通过率

In [25]:
#### 申请表通过率

In [26]:
accept_rate_dairly = shouxin_monitor_dairly[['score_interval']]
for i in range(len(shouxin_monitor_dairly.columns)):
    if(i>=2 and i <=7):
        shouxin_col = shouxin_monitor_dairly.columns[i]
        apply_col = apply_monitor_dairly.columns[i]
#         print(shouxin_col,apply_col)

        accept_col = shouxin_col.split("_")[1]+"_"+shouxin_col.split("_")[2]
        accept_rate_dairly[accept_col+"_rate"] = shouxin_monitor_dairly[shouxin_col]/(apply_monitor_dairly[apply_col])
        accept_rate_dairly[accept_col+"_rate"][6] = apply_monitor_dairly[apply_col][6]

accept_rate_dairly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://p

,score_interval,uid_month_rate,applyid_month_rate,uid_week_rate,applyid_week_rate,uid_today_rate,applyid_today_rate
0,>680,0.400000,0.400000,0.250000,0.250000,NaN,NaN
1,"(650,680]",0.606061,0.606061,0.571429,0.571429,0.0,0.0
2,"(630,650]",0.405063,0.405063,0.560976,0.560976,0.0,0.0
3,"(610,630]",0.302013,0.302013,0.328947,0.328947,0.0,0.0
4,"(595,610]",0.096257,0.096257,0.132812,0.132812,0.0,0.0
5,<=595,0.002433,0.002430,0.001621,0.001621,0.0,0.0
6,汇总,1313.000000,1314.000000,905.000000,905.000000,28.0,28.0


In [27]:
shouxin_monitor_monthly

,score_interval,toutiao_develop,shouxin_uid_online,shouxin_applyid_online,shouxin_uid_2018_07,shouxin_applyid_2018_07,shouxin_uid_2018_08,shouxin_applyid_2018_08,shouxin_uid_2018_09,shouxin_applyid_2018_09,...,shouxin_uid_2018_09_percent,shouxin_applyid_2018_09_percent,shouxin_uid_2018_10_percent,shouxin_applyid_2018_10_percent,shouxin_uid_2018_11_percent,shouxin_applyid_2018_11_percent,shouxin_uid_2018_12_percent,shouxin_applyid_2018_12_percent,shouxin_uid_2019_01_percent,shouxin_applyid_2019_01_percent
0,>680,24.89%,4.0,4.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028369,0.028369
1,"(650,680]",20.10%,40.0,40.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.283688,0.283688
2,"(630,650]",17.00%,32.0,32.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.226950,0.226950
3,"(610,630]",14.87%,45.0,45.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.319149,0.319149
4,"(595,610]",14.29%,18.0,18.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.127660,0.127660
5,<=595,10.85%,2.0,2.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014184,0.014184
6,汇总,21590,141.0,141.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.000000,141.000000


In [28]:
apply_monitor_monthly

,score_interval,toutiao_develop,apply_uid_online,apply_sessionId_online,apply_uid_2018_07,apply_sessionId_2018_07,apply_uid_2018_08,apply_sessionId_2018_08,apply_uid_2018_09,apply_sessionId_2018_09,...,apply_uid_2018_09_percent,apply_sessionId_2018_09_percent,apply_uid_2018_10_percent,apply_sessionId_2018_10_percent,apply_uid_2018_11_percent,apply_sessionId_2018_11_percent,apply_uid_2018_12_percent,apply_sessionId_2018_12_percent,apply_uid_2019_01_percent,apply_sessionId_2019_01_percent
0,>680,24.89%,10.0,10.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007457,0.007452
1,"(650,680]",20.10%,70.0,70.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052200,0.052161
2,"(630,650]",17.00%,82.0,82.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.061148,0.061103
3,"(610,630]",14.87%,152.0,152.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.113348,0.113264
4,"(595,610]",14.29%,191.0,191.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142431,0.142325
5,<=595,10.85%,836.0,837.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.623415,0.623696
6,汇总,21590,1341.0,1342.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1341.000000,1342.000000


In [29]:
accept_rate_monthly = shouxin_monitor_monthly[['score_interval']]
for i in range(len(shouxin_monitor_monthly.columns)):
    if(i>=2 and i <=16):
        shouxin_col = shouxin_monitor_monthly.columns[i]
        apply_col = apply_monitor_monthly.columns[i]
#       print(shouxin_col,apply_col)

        accept_col = shouxin_col[8:]
        accept_rate_monthly[accept_col+"_rate"] = shouxin_monitor_monthly[shouxin_col]/(apply_monitor_monthly[apply_col])
        accept_rate_monthly[accept_col+"_rate"][6] = apply_monitor_monthly[apply_col][6]

accept_rate_monthly = accept_rate_monthly.fillna("0")
accept_rate_monthly


/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,score_interval,uid_online_rate,applyid_online_rate,uid_2018_07_rate,applyid_2018_07_rate,uid_2018_08_rate,applyid_2018_08_rate,uid_2018_09_rate,applyid_2018_09_rate,uid_2018_10_rate,applyid_2018_10_rate,uid_2018_11_rate,applyid_2018_11_rate,uid_2018_12_rate,applyid_2018_12_rate,uid_2019_01_rate
0,>680,0.400000,0.400000,0,0,0,0,0,0,0,0,0,0,0,0,0.400000
1,"(650,680]",0.571429,0.571429,0,0,0,0,0,0,0,0,0,0,0,0,0.571429
2,"(630,650]",0.390244,0.390244,0,0,0,0,0,0,0,0,0,0,0,0,0.390244
3,"(610,630]",0.296053,0.296053,0,0,0,0,0,0,0,0,0,0,0,0,0.296053
4,"(595,610]",0.094241,0.094241,0,0,0,0,0,0,0,0,0,0,0,0,0.094241
5,<=595,0.002392,0.002389,0,0,0,0,0,0,0,0,0,0,0,0,0.002392
6,汇总,1341.000000,1342.000000,0,0,0,0,0,0,0,0,0,0,0,0,1341.000000


In [30]:
##### 头条征信评分相关报告